张量
- view
- reshape
- transpose 转置
- permute

In [4]:
import torch
x = torch.arange(1, 4)
x = x.reshape(3,1)
print(x)

tensor([[1],
        [2],
        [3]])


- 升维
torch.unsqueeze(input, dim, out=None)
- 降维
torch.squeeze(input, dim=None, out=None)

In [6]:
a = torch.tensor([1, 2, 3, 4])
a.shape

torch.Size([4])

In [7]:
b = torch.unsqueeze(a, dim=0)
print(b, b.shape)

tensor([[1, 2, 3, 4]]) torch.Size([1, 4])


加载数据
- Dataset：加载数据集
- DataLoader：迭代访问数据集的样本


In [9]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

100%|██████████| 26421880/26421880 [00:02<00:00, 10394543.75it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 155472.38it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:02<00:00, 1825109.45it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 32765215.47it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [10]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

# 灰度图：单通道，1
# 分辨率：28 * 28
# 此前一个batch的大小：64
img = train_features[0].squeeze()
label = train_labels[0]
print(img.shape)
print(f"Label: {label}")

Feature batch shape: torch.Size([64, 1, 28, 28])
Labels batch shape: torch.Size([64])
torch.Size([28, 28])
Label: 7


训练模型

In [11]:
import torch
from torch import nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [12]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), # hidden
            nn.ReLU(),
            nn.Linear(512, 256), # hidden
            nn.ReLU(),
            nn.Linear(256, 10), # output layer, 10 outputs
            nn.Dropout(p=0.2) # alleviate over-fitting
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=10, bias=True)
    (5): Dropout(p=0.2, inplace=False)
  )
)


In [13]:
X = torch.rand(4, 28, 28, device=device)
logits = model(X)
# 输出logits，接一个Softmax层
pred_probab = nn.Softmax(dim=1)(logits)
print(pred_probab.size())
# 每个样本输出一个 10 维的概率向量

torch.Size([4, 10])


In [14]:
y_pred = pred_probab.argmax(-1)
print(f"Predicted class: {y_pred}")
# 将输出的概率向量转换为对应的标签。

Predicted class: tensor([6, 2, 2, 6], device='cuda:0')


模型参数
1. 损失函数
2. 优化器

In [15]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader, start=1):
        X, y = X.to(device), y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y) # 自动计算loss
        # Backpropagation
        optimizer.zero_grad() # 防止梯度累计，每个训练阶段都要清零
        loss.backward()  # 反向传播
        optimizer.step() # 调整参数

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    model.eval() # 评估模式，dropout层失活
    with torch.no_grad():  # 关闭自动梯度，不更新参数
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item() 
            correct += (pred.argmax(dim=-1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [16]:
# 超参数
learning_rate = 1e-3
batch_size = 64
epochs = 3

# 数据加载
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [17]:
# 交叉熵损失函数
loss_fn = nn.CrossEntropyLoss()
# AdamW优化器，随着训练过程逐步减小学习率
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.888740  [ 6400/60000]
loss: 0.877878  [12800/60000]
loss: 0.632524  [19200/60000]
loss: 0.889715  [25600/60000]
loss: 0.346180  [32000/60000]
loss: 0.585663  [38400/60000]
loss: 0.892315  [44800/60000]
loss: 0.624809  [51200/60000]
loss: 0.508609  [57600/60000]
Test Error: 
 Accuracy: 84.2%, Avg loss: 0.445821 

Epoch 2
-------------------------------
loss: 0.587900  [ 6400/60000]
loss: 0.739247  [12800/60000]
loss: 0.648173  [19200/60000]
loss: 0.689126  [25600/60000]
loss: 0.496668  [32000/60000]
loss: 0.542248  [38400/60000]
loss: 0.857910  [44800/60000]
loss: 0.517630  [51200/60000]
loss: 0.557837  [57600/60000]
Test Error: 
 Accuracy: 86.1%, Avg loss: 0.386906 

Epoch 3
-------------------------------
loss: 0.607352  [ 6400/60000]
loss: 0.547249  [12800/60000]
loss: 0.433128  [19200/60000]
loss: 0.698187  [25600/60000]
loss: 0.382055  [32000/60000]
loss: 0.525773  [38400/60000]
loss: 0.743243  [44800/60000]
loss: 0.454474  [51200/600

保存/加载模型

In [ ]:
# 权重：state_dict
import torch
import torchvision.models as models

model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
model = models.vgg16() # we do not specify pretrained=True, i.e. do not load default weights
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

In [18]:
# 模型架构和权重
import torch
import torchvision.models as models

model = models.vgg16(pretrained=True)
torch.save(model, 'model.pth')

/home/hesirui/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hesirui/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/hesirui/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:29<00:00, 18.6MB/s] 


In [21]:
model = torch.load('model.pth')
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

评估性能

In [22]:
from sklearn.metrics import classification_report

y_true = [1, 1, 0, 1, 2, 1, 0, 2, 1, 1, 0, 1, 0]
y_pred = [1, 0, 0, 1, 2, 0, 1, 1, 1, 0, 0, 1, 0]

print(classification_report(y_true, y_pred, output_dict=False))

              precision    recall  f1-score   support

           0       0.50      0.75      0.60         4
           1       0.67      0.57      0.62         7
           2       1.00      0.50      0.67         2

    accuracy                           0.62        13
   macro avg       0.72      0.61      0.63        13
weighted avg       0.67      0.62      0.62        13

